# To join or not to join
* Merging datasets

**Instructions:** This is an individual assignment, but you may discuss your code with your classmates.

**Problem type key and definition:**
* _Exercises_ are autograded on GitHub classroom
* _Problems_ are manually graded and often open ended without a single correct answer.
* _Stop and think_ prompts are not graded, and are provided to guide you.

Please see the README for instructions on how to submit and obtain the lab.

In [13]:
%load_ext autoreload
%autoreload 2


# Put all your solutions into Lab1_helper.py as this script which is autograded
import Lab4_helper 

from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. 
# This is not relevant to most people because I recommended you use my server, but
# change home to where you are storing everything. Again. Not recommended.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data
There are a lot of different data files associated with this pesticide data:

>This dataset contains information on pesticide residues in food. The U.S. Department of Agriculture (USDA) Agricultural Marketing Service (AMS) conducts the Pesticide Data Program (PDP) every year to help assure consumers that the food they feed themselves and their families is safe. Ultimately, if EPA determines a pesticide is not safe for human consumption, it is removed from the market.

>The PDP tests a wide variety of domestic and imported foods, with a strong focus on foods
that are consumed by infants and children. EPA relies on PDP data to conduct dietary risk
assessments and to ensure that any pesticide residues in foods remain at safe levels. USDA
uses the data to better understand the relationship of pesticide residues to agricultural practices
and to enhance USDA’s Integrated Pest Management objectives. USDA also works with U.S.
growers to improve agricultural practices.

> While the original 2013 MS Access database can be found here, the data has been transferred to a SQLite database for easier, more open use. The database contains two tables, Sample Data and Results Data. Each sampling includes attributes such as extraction method, the laboratory responsible for the test, and EPA tolerances among others. These attributes are labeled with codes, which can be referenced in PDF format here, or integrated into the database using the included csv files.

Source: https://www.kaggle.com/usdeptofag/pesticide-data-program-2013

We will first examine files that define codes and relationships. They are stored as a dictionary of dataframes with the key being the file name without the .csv.

In [14]:
import pandas as pd
import glob
dfs = {}
for file in glob.glob(f'{home}/data-301-student/data/pesticide/*.csv'):
    name = file.split("/")[-1].replace(".csv","")
    dfs[name] = pd.read_csv(file)
    print(name)
    display(dfs[name].head())

anotate_codes


,Annotate Code,Annotated Information
0,Q,Residue at below quantifiable level (BQL)
1,QV,Residue at <BQL> with presumptive violation - ...
2,V,Residue with a presumptive violation - No Tole...
3,X,Residue with a presumptive violation - Exceeds...


commodity_codes


,Commodity Code,Commodity Name,# of Samples Analyzed
0,AP,Apples,177
1,BB,"Blueberries, Cultivated",688
2,BN,Bananas,179
3,BR,Broccoli,712
4,BZ,"Blueberries, Frozen",19


tolerance_codes


...

,ConfMethod Code,Confirmation Method
0,GT,GC/MS/MS - triple quadropole
1,LU,LC-MS/MS - triple quadrapole
2,MO,Quant. & Confirm. by GC/MS only


determin_codes


,Determin Code,Determinative Method
0,7,GC/MS - Gas Chrom w/Mass Spec - single quadrupole
1,35,GC/MS/MS - triple quadrupole
2,52,LC/MS/MS - Liquid Chrom w/ Tandem Mass Spec - ...
3,64,Second LC/MS/MS
4,67,Third LC/MS/MS


extract_codes


,Extract Code,Extraction Method
0,805,MDA Modified QuEChERS Method
1,818,NSL Animal Tissue Extraction Method
2,823,WSDA Animal Tissue Extraction Method
3,998,OTHER Single-Analysis Methods


We will now read in data itself.

In [15]:
import pandas as pd
import glob
data = {}
for file in glob.glob(f'{home}/data-301-student/data/pesticide/*.csv.gz'):
    name = file.split("/")[-1].replace(".csv.gz","")
    data[name] = pd.read_csv(file)
    print(name)
    display(data[name].head())

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


resultsdata13


,sample_pk,commod,commtype,lab,pestcode,testclass,concen,lod,conunit,confmethod,confmethod2,annotate,quantitate,mean,extract,determin
0,1,AJ,RE,WA1,540,A,NaN,0.002,M,NaN,NaN,NaN,NaN,ND,805,35
1,1,AJ,RE,WA1,562,C,NaN,0.001,M,NaN,NaN,NaN,NaN,ND,805,52
2,1,AJ,RE,WA1,594,F,NaN,0.005,M,NaN,NaN,NaN,NaN,ND,805,52
3,1,AJ,RE,WA1,596,A,NaN,0.002,M,NaN,NaN,NaN,NaN,ND,805,52
4,1,AJ,RE,WA1,597,O,NaN,0.010,M,NaN,NaN,NaN,NaN,ND,805,35


sampledata13


,sample_pk,state,year,month,day,site,commod,source_id,variety,origin,country,disttype,commtype,claim,quantity,growst,packst,distst
0,1,CA,13,1,22,4,AJ,P,Apple Juice,2,150,R,RE,PO,NaN,NaN,NaN,IL\r\n
1,2,CA,13,1,22,150,AJ,P,NaN,1,NaN,R,RE,NC,NaN,NaN,NaN,CA\r\n
2,3,CA,13,1,22,151,AJ,P,Unknown,2,M68,R,RE,NC,NaN,NaN,NaN,CA\r\n
3,4,CA,13,1,22,273,AJ,P,Apple Juice,2,MH8,R,RE,NC,NaN,NaN,NaN,CA\r\n
4,5,CA,13,1,22,328,AJ,P,NaN,1,NaN,R,RE,NC,NaN,NaN,NaN,CA\r\n


In [16]:
data['resultsdata13'].sort_values(by='concen',ascending=False)

,sample_pk,commod,commtype,lab,pestcode,testclass,concen,lod,conunit,confmethod,confmethod2,annotate,quantitate,mean,extract,determin
1904954,9637,WG,FR,MN1,ACG,A,1308.0,3.0000,T,LU,NaN,NaN,NaN,O,999,52
1961225,9888,WU,FR,MT1,ACH,A,1200.0,1.8000,T,LU,NaN,NaN,NaN,O,811,52
1907283,9651,WR,FR,MT1,ACH,A,1100.0,1.8000,T,LU,NaN,NaN,NaN,O,811,52
1961579,9890,WU,FR,MT1,ACH,A,1100.0,1.8000,T,LU,NaN,NaN,NaN,O,811,52
1961402,9889,WU,FR,MT1,ACH,A,1100.0,1.8000,T,LU,NaN,NaN,NaN,O,811,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023082,10104,YF,CO,CA1,B78,L,NaN,0.0100,M,NaN,NaN,NaN,NaN,ND,805,52
2023083,10104,YF,CO,CA1,B79,F,NaN,0.0025,M,NaN,NaN,NaN,NaN,ND,805,52
2023084,10104,YF,CO,CA1,B80,A,NaN,0.0025,M,NaN,NaN,NaN,NaN,ND,805,52
2023085,10104,YF,CO,CA1,B84,F,NaN,0.0025,M,NaN,NaN,NaN,NaN,ND,805,52


## What are the most common types of pesticides tested in this study?

**Exercise 1:** The dataframe stored in ``data['resultsdata13']['pestcode']`` has test pesticide test results. Write a function that counts the number of times a pesticide is tested and join these results to ``dfs['pest_codes']``. Report this count with the pestecide name and sort by pestecide name. Here is the example output:

In [17]:
df = Lab4_helper.exercise_1(data['resultsdata13'],dfs['pest_codes'])
df

,Pesticide Name,Count
140,1-Naphthol,3234
125,"2,4,5-T",114
7,"2,4-D",114
126,"2,4-DB",114
146,3-Hydroxycarbofuran,10058
...,...,...
323,Triticonazole,3242
382,Uniconazole,1943
79,Vernolate,1307
148,Vinclozolin,8344


## Do certain states tend to use one particular pesticide type over another?

**Exercise 2:** Return a dataframe that lists the top pesticide by state. Sort by state.

In [18]:
df = Lab4_helper.exercise_2(data['sampledata13'],data['resultsdata13'],dfs['pest_codes'])
df

,State,Pesticide Name
0,CA,Heptachlor
3,CO,Dieldrin
4,FL,Dieldrin
8,KY,Diazinon
11,LA,Aldrin
5,MD,Dieldrin
6,MI,Dieldrin
9,MN,Diazinon
12,MO,Aldrin
13,NC,Aldrin


## How consistent are the top pesticides per commodity? How much does this change from state to state?

In [19]:
dfs['commodity_codes'].head()

,Commodity Code,Commodity Name,# of Samples Analyzed
0,AP,Apples,177
1,BB,"Blueberries, Cultivated",688
2,BN,Bananas,179
3,BR,Broccoli,712
4,BZ,"Blueberries, Frozen",19


**Exercise 3:** Return a dataframe that lists the top pesticide by commodity. Sort by Commodity Name.

In [20]:
df = Lab4_helper.exercise_3(data['sampledata13'],data['resultsdata13'],dfs['pest_codes'],dfs['commodity_codes'])
df

,Commodity Name,Pesticide Name
0,Bananas,Aldrin
10,Broccoli,Diazinon
2,Carrots,Aldrin
1,Celery,Aldrin
3,"Fish, Salmon",Aldrin
5,Grape Juice,Aldrin
4,Green Beans,Aldrin
11,"Infant Formula, Dairy",Dieldrin
9,"Infant Formula, Soy-based",Aldrin
6,Nectarines,Aldrin


**Exercise 4:** Return a dataframe that lists the top pesticide by commodity and state. Sort by commodity name.

In [21]:
df = Lab4_helper.exercise_4(data['sampledata13'],data['resultsdata13'],dfs['pest_codes'],dfs['commodity_codes'])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

                Commodity Name State Pesticide Name
0                      Bananas    CA         Aldrin
1                      Bananas    CO         Aldrin
2                      Bananas    FL         Aldrin
3                      Bananas    MD         Aldrin
4                      Bananas    MI         Aldrin
5                      Bananas    NY         Aldrin
6                      Bananas    OH         Aldrin
7                      Bananas    TX         Aldrin
8                      Bananas    WA         Aldrin
9                      Bananas    WI         Aldrin
10                    Broccoli    CA         Aldrin
11                    Broccoli    CO         Aldrin
12                    Broccoli    FL         Aldrin
13                    Broccoli    MD         Aldrin
14                    Broccoli    MI         Aldrin
106                   Broccoli    NY       Diazinon
15                    Broccoli    OH         Aldrin
16                    Broccoli    TX         Aldrin
17          

**Problem 1:** Are there any exceptions to the top pesticide per commodity across states? If so, what are they?

**Your answer here**

**Problem 2:** Come up with three more questions you would want to know from this data? You should be able to answer them from this data. i.e., you must avoid asking questions where more data is needed. 

**Your answer here**

In [22]:
# Good job!
# Don't forget to push with ./submit.sh